### Gemini Pro 기본 동작 예제

Feedback : shins777@gmail.com

이 Colab은 Gemini Pro 의 기본 동작을 이해하기 위한 내용입니다.  
Langchain을 이용해서 간단하게 Generative AI가 동작하는 방식을 나타냅니다.

#라이브러리 설치
상세한 Python API 정보는 아래 URL 참고하세요.
*   Langchain library : https://github.com/langchain-ai/langchain
*   Langchain Vertex AI API : https://api.python.langchain.com/en/stable/google_vertexai_api_reference.html



In [ ]:
!pip install --upgrade --quiet langchain langchain-core langchain-google-vertexai

### GCP 사용자 인증 / 환경설정

Colab 환경에서는 아래와 같이 인증이 필요합니다.

In [ ]:
#  아래 코드는 Colab 환경에서만 실행해주세요.
from google.colab import auth
auth.authenticate_user()

In [2]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_google_vertexai.llms import VertexAI

2024-03-10 01:08:49.045994: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-10 01:08:49.876742: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-10 01:08:49.876848: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-10 01:08:50.064754: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-10 01:08:50.439985: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-10 01:08:50.443479: I tensorflow/core/platform/cpu_feature_guard.cc:1

본인의 GCP 환경에 맞게 아래 설정을 구성하세요. 단, model_name은 "gemini-pro" 를 그대로 사용하세요.

In [3]:
model_name="gemini-pro"
project="ai-hangsik"
location="asia-northeast3"

# 모델 초기화 및 기본 실행

구글은 아래와 같이 생성형 AI 처리시 Responsible AI 부분으로 Harmcategory 에 따른 LLM 응답 조절이 가능합니다.


Responsible AI setting
*   HarmCategory : https://cloud.google.com/vertex-ai/docs/reference/rest/v1/HarmCategory
*   HarmBlockThreshold : https://cloud.google.com/php/docs/reference/cloud-ai-platform/0.31.0/V1.SafetySetting.HarmBlockThreshold



In [4]:
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory

safety_settings = {
                    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
}

*   VertexAI API : https://api.python.langchain.com/en/stable/llms/langchain_google_vertexai.llms.VertexAI.html#langchain_google_vertexai.llms.VertexAI

*  https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/gemini#gemini-pro

아래 내용은 Langchain 기반에서의 gemini_pro 를 초기화하는 방법입니다.
업무에 따라서 Langchain 을 사용하셔도 좋고, 만일 latency 가 중요하다고 하면, Google 자체 API를 사용하는것도 Latency를 줄이는 방법이 될수 있습니다.

In [5]:
gemini_pro = VertexAI( model_name=model_name,
                  project=project,
                  location=location,
                  verbose=True,
                  streaming=True,
                  safety_settings = safety_settings,
                  temperature = 0.2,
                  top_p = 1,
                  top_k = 10
                 )

In [6]:
context ="""
광복 후 70년을 맞은 2015년 한국은 전환기를 맞고 있다. 고령화사회(65세 이상 인구가 14% 이상)로 다가가고 있는 우리나라는 2060년이 되면 고령인구 비중이 40.1%가 돼 세계에서 두 번째로 늙은 나라가 될 것이라는 전망이 나온다. 출산율은 1.23명(2010∼2014년 평균)으로 세계에서 네 번째로 낮다. 그만큼 경제활동을 할 수 있는 인구가 적어진다는 뜻이다.
한국 경제가 이제 저성장 국면으로 접어든 것이 아니냐는 우려도 나온다. 1961년부터 50년간 한국의 연간 경제성장률이 3%를 밑돈 것은 다섯 차례로 오일쇼크, 외환위기, 카드사태, 글로벌 금융위기 등의 특수한 이유가 있었다. 그마저도 충격을 딛고 1~2년 안에 경제가 빠른 반등에 성공했다. 하지만 2012년부터는 이렇다 할 외부충격이 없는데도 작년(3.3%)을 제외하고는 계속 2%대의 낮은 성장세를 보이고 있다.
‘제2의 도약’을 위해서는 한계점에 다다른 기존 산업의 성장, 비정규직·정규직으로 분화된 노동시장, 세대갈등, 청년실업 등 풀어야 할 문제가 한둘이 아니다. 외환위기 직후 재정경제부 수장으로 경제정책을 이끈 강봉균 전 장관은 “한국은 과감한 개방 등 다른 개발도상국이 못하던 것들을 해내 선진국 문턱으로 올라설 수 있었다”며 “선진국들이 힘들어하는 구조개혁에 성공하면 다시 한번 도약할 수 있을 것”이라고 말했다."""

aspect = "문제점"

prompt = PromptTemplate.from_template("아래 Context 에 대해서 {aspect} 관점에서 요약 해주세요. Context : {context} ")
prompt = prompt.format(aspect=aspect,
                       context=context)

print(gemini_pro.invoke(prompt))


**문제점 요약**

* **고령화:** 2060년까지 고령인구 비중이 40.1%로 세계에서 두 번째로 높아질 것으로 예상됨.
* **저출산율:** 세계에서 네 번째로 낮은 출산율(1.23명)로 경제활동 인구 감소 우려.
* **저성장 경제:** 2012년 이후 외부 충격 없이도 2%대의 낮은 성장세 지속.
* **기존 산업의 성장 한계:** 기존 산업의 성장이 한계에 도달함.
* **노동시장 분화:** 비정규직과 정규직으로 분화된 노동시장으로 인한 불평등 심화.
* **세대갈등:** 세대 간의 가치관 및 경제적 이익 차이로 인한 갈등.
* **청년실업:** 청년층의 높은 실업률로 인한 경제적 불안정.


# Basic Operation
1. 문장에서 Fact 를 요약하는 예제
2. 두개의 명령을 한번 호출로 처리하는 예제.

*   PromptTemplate: https://api.python.langchain.com/en/stable/langchain_api_reference.html#module-langchain.prompts
*   class hierarchy :
https://screenshot.googleplex.com/8JuER9DU4rqqsqE.png



In [7]:
context1 ="""
광복 후 70년을 맞은 2015년 한국은 전환기를 맞고 있다. 고령화사회(65세 이상 인구가 14% 이상)로 다가가고 있는 우리나라는 2060년이 되면 고령인구 비중이 40.1%가 돼 세계에서 두 번째로 늙은 나라가 될 것이라는 전망이 나온다. 출산율은 1.23명(2010∼2014년 평균)으로 세계에서 네 번째로 낮다. 그만큼 경제활동을 할 수 있는 인구가 적어진다는 뜻이다.
한국 경제가 이제 저성장 국면으로 접어든 것이 아니냐는 우려도 나온다. 1961년부터 50년간 한국의 연간 경제성장률이 3%를 밑돈 것은 다섯 차례로 오일쇼크, 외환위기, 카드사태, 글로벌 금융위기 등의 특수한 이유가 있었다. 그마저도 충격을 딛고 1~2년 안에 경제가 빠른 반등에 성공했다. 하지만 2012년부터는 이렇다 할 외부충격이 없는데도 작년(3.3%)을 제외하고는 계속 2%대의 낮은 성장세를 보이고 있다.
‘제2의 도약’을 위해서는 한계점에 다다른 기존 산업의 성장, 비정규직·정규직으로 분화된 노동시장, 세대갈등, 청년실업 등 풀어야 할 문제가 한둘이 아니다. 외환위기 직후 재정경제부 수장으로 경제정책을 이끈 강봉균 전 장관은 “한국은 과감한 개방 등 다른 개발도상국이 못하던 것들을 해내 선진국 문턱으로 올라설 수 있었다”며 “선진국들이 힘들어하는 구조개혁에 성공하면 다시 한번 도약할 수 있을 것”이라고 말했다."""

context2 = """
한국 헌법은 “모든 국민의 재산권은 보장된다”라고 명시하고 있다. 시장 경제 체제를 기본으로 하 고, 개인과 기업의 자유로운 경제 활동을 존중하며, 그로써 얻은 개인과 기업의 이득과 재산을 모 두 보장한다.
그러나 헌법은 자본주의적 시장 경제를 무제한 보장하지 않는다. 헌법은 자본을 지나치게 남용해 타인에게 피해를 주면 이를 조정할 수 있도록 규정하고 있다. 이는 자유 시장 경제의 문제점을 보완 하기 위한 최소한의 장치이다."""

prompt = PromptTemplate(
    input_variables=["text_input1", "text_input2"],
    template="""이 텍스트에서 핵심 사실을 추출해 보세요. 의견을 포함하지 마세요. 각 사실에 대해서 상세하게 설명해주세요.  : {text_input1}
    그리고 아래 context: 부분을 요약해주세요. context : {text_input2}
    """
)

fact_extraction_chain = LLMChain(llm=gemini_pro, prompt=prompt, verbose=True)
facts = fact_extraction_chain.run(text_input1=context1,text_input2=context2 )

print(facts)


/home/admin_/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
이 텍스트에서 핵심 사실을 추출해 보세요. 의견을 포함하지 마세요. 각 사실에 대해서 상세하게 설명해주세요.  : 
광복 후 70년을 맞은 2015년 한국은 전환기를 맞고 있다. 고령화사회(65세 이상 인구가 14% 이상)로 다가가고 있는 우리나라는 2060년이 되면 고령인구 비중이 40.1%가 돼 세계에서 두 번째로 늙은 나라가 될 것이라는 전망이 나온다. 출산율은 1.23명(2010∼2014년 평균)으로 세계에서 네 번째로 낮다. 그만큼 경제활동을 할 수 있는 인구가 적어진다는 뜻이다.
한국 경제가 이제 저성장 국면으로 접어든 것이 아니냐는 우려도 나온다. 1961년부터 50년간 한국의 연간 경제성장률이 3%를 밑돈 것은 다섯 차례로 오일쇼크, 외환위기, 카드사태, 글로벌 금융위기 등의 특수한 이유가 있었다. 그마저도 충격을 딛고 1~2년 안에 경제가 빠른 반등에 성공했다. 하지만 2012년부터는 이렇다 할 외부충격이 없는데도 작년(3.3%)을 제외하고는 계속 2%대의 낮은 성장세를 보이고 있다.
‘제2의 도약’을 위해서는 한계점에 다다른 기존 산업의 성장, 비정규직·정규직으로 분화된 노동시장, 세대갈등, 청년실업 등 풀어야 할 문제가 한둘이 아니다. 외환위기 직후 재정경제부 수장으로 경제정책을 이끈 강봉균 전 장관은 “한국은 과감한 개방 등 다른 개발도상국이 못하던 것들을 해내 선진국 문턱으로 올라설 수 있었다”며 “선진국들이 힘들어하는 구조개혁에 성공하면 다시 한번 도약할 수 있을 것”이라고 말했다.
    그리고 아래 context: 부분을 요약해주세요. context : 
한국 헌법은 “모든 국민의 재산권은 보장된다”라고 명시하고 있다. 시장 경제 체제를 기본으로 하 고, 개인과 기업의 자유로운 경제 활동을 존중하며, 그로써 얻은 개인과 기업의 이득과 재산을 모 두 보장한다.
그러나 헌법은 자본주의적 시장 경제를 무제한 보장하지 않는다. 헌법은

# SequentialChain
1. 두개의 연관된 질문을 한번의 호출로 연계.
2. 이전 답변에서 사실을 추출해서 다음 질문에 활용할 때 사용.

*   LLMChain : https://api.python.langchain.com/en/stable/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain



In [8]:
from langchain.prompts import PromptTemplate

prompt_1 = PromptTemplate(
    input_variables=["country"],
    template="{country}의 수도는 어디인가요 ?",
)
capital_chain = LLMChain(llm=gemini_pro, prompt=prompt_1, verbose=True, output_key="capital")

prompt_2 = PromptTemplate(
    input_variables=["capital"],
    template=" {capital} 에서 가장 유명한 곳 10 군데를 알려주세요.",
)
places_chain = LLMChain(llm=gemini_pro, prompt=prompt_2, verbose=True, output_key="places")


In [9]:
from langchain.chains import SequentialChain

# https://api.python.langchain.com/en/latest/chains/langchain.chains.sequential.SequentialChain.html#

overall_chain : SequentialChain = SequentialChain(
      chains=[capital_chain, places_chain],
      input_variables=["country"], # 이곳에 처음 입력 받는 변수를 정의.
      output_variables=["capital", "places"], # 이곳에 여러개 return 되는 변수를 정의.
      verbose=True
    )

output_from_ai : dict = overall_chain({"country":"대한민국"})

for key, value in output_from_ai.items():
  print(value)


/home/admin_/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
대한민국의 수도는 어디인가요 ?

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
 서울 에서 가장 유명한 곳 10 군데를 알려주세요.

> Finished chain.

> Finished chain.
대한민국
서울
**서울에서 가장 유명한 곳 10곳**

1. **경복궁:** 조선 왕조의 주요 궁전으로, 웅장한 건축물과 아름다운 정원으로 유명합니다.
2. **창덕궁:** 유네스코 세계 문화 유산으로 지정된 또 다른 궁전으로, 화려한 건축물과 평화로운 정원으로 유명합니다.
3. **남산타워:** 서울의 상징적인 랜드마크로, 도시의 탁 트인 전경을 제공합니다.
4. **명동:** 번화한 쇼핑 및 엔터테인먼트 지구로, 다양한 상점, 레스토랑, 카페가 있습니다.
5. **인사동:** 전통 예술품, 골동품, 공예품을 판매하는 매력적인 골목길입니다.
6. **광장시장:** 서울에서 가장 큰 전통 시장으로, 다양한 상품과 맛있는 길거리 음식을 판매합니다.
7. **한강공원:** 한강을 따라 위치한 대규모 공원으로, 산책, 자전거 타기, 피크닉을 즐길 수 있습니다.
8. **국립중앙박물관:** 한국 역사와 문화에 관한 광범위한 유물을 소장하고 있는 세계적인 박물관입니다.
9. **동대문디자인플라자(DDP):** 독특한 디자인과 첨단 기술로 유명한 문화 및 디자인 센터입니다.
10. **롯데월드:** 대규모 놀이공원으로, 다양한 놀이기구, 공연, 쇼핑몰을 갖추고 있습니다.
